In [152]:
import pandas as pd
import os
import re
import requests
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import tqdm

In [167]:
driver = webdriver.Chrome('./chromedriver')

/Users/gunwook/opt/anaconda3/envs/gunwook/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


In [168]:
# 여기서 무조건 로그인 해야함. 
driver.get('https://nid.naver.com/nidlogin.login?svctype=262144&url=http://naver.com/')

In [156]:
driver.find_element_by_xpath('//*[@id="menuLink0"]').click()

/Users/gunwook/opt/anaconda3/envs/gunwook/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  """Entry point for launching an IPython kernel.


In [157]:
def crawl_mbti_data():
    pg = str(0)
    
    for i in tqdm.tqdm(range(1, 1000)):
        pg = str(i)
        tmp_df = pd.DataFrame()
        tmp_category = []
        tmp_writer = []
        tmp_title = []
        tmp_link = []
        
        addr = "https://cafe.naver.com/ArticleList.nhn?search.clubid=11856775&userDisplay=50&search.boardtype=L&search.specialmenutype=&search.totalCount=501&search.cafeId=11856775&search.page=" + pg            

        driver.get(addr)
        driver.switch_to.frame('cafe_main')
        
        html = driver.page_source

        soup = bs(html, 'html.parser')
        
        a_category_list = soup.findAll("div", {"class": "inner_name"})
        a_title_list = soup.findAll("a",{"class":"article"})
        a_writer_list = soup.findAll("a",{"class":"m-tcol-c"}) 
        
        if pg == '1':
            a_title_list = a_title_list[16:]
            a_writer_list = a_writer_list[16:]
            
        for j in range(0, len(a_title_list)):
            tmp_writer.append(a_writer_list[j].text.strip())
            tmp_title.append(a_title_list[j].text.strip())
            tmp_category.append(a_category_list[j].text.strip())
            tmp_link.append("https://cafe.naver.com/mbticafe" + a_title_list[j].get('href').strip())
        
        tmp_df['category'] = tmp_category
        tmp_df['title'] = tmp_title
        tmp_df['writer'] = tmp_writer
        tmp_df['link'] = tmp_link
        
        tmp_df.to_csv(f'./crawl_data/crawl_res_{i}.csv', index=False)
            
            

In [149]:
crawl_mbti_data()

100%|████████████████████████████████| 999/999 [17:30<00:00,  1.05s/it]


In [150]:
def crawl_article(path):
    file_list = os.listdir(path)
    for file in tqdm.tqdm(file_list):
        file_num = file.split('.')[0].split('_')[-1]
        df = pd.read_csv(path + '/' + file)
        tmp_df = pd.DataFrame()
        
        row_len = len(df['category'])
        tmp_article = []
        tmp_category = []
        tmp_writer = []
        tmp_title = []
        tmp_link = []
        for r_idx in range(0, row_len):
            try:
                row = df.iloc[r_idx]
                if row['category'] == '가입 인사 (등업 필수)':
                    continue

                driver.get(row['link'])
                driver.switch_to.frame('cafe_main')
                html = driver.page_source
                soup = bs(html, 'html.parser')
                article = soup.findAll("div", {"class": "tbody m-tcol-c"})[0].text.replace('\u200b', ' ').replace('\n','').strip()
                if article.strip() == '':
                    continue

                tmp_article.append(article)
                tmp_category.append(row['category'])
                tmp_writer.append(row['writer'])
                tmp_link.append(row['link'])
                tmp_title.append(row['title'])
            except:
                continue
        
        tmp_df['category'] = tmp_category
        tmp_df['title'] = tmp_title
        tmp_df['writer'] = tmp_writer
        tmp_df['link'] = tmp_link
        tmp_df['article'] = tmp_article
        
        tmp_df.to_csv(f'./crawl_data/crawl_res_{file_num}_fin.csv', index=False)
                
            

In [151]:
path = '/Users/gunwook/Desktop/git_folder/mbti_prediction/crawl/crawl_data'
crawl_article(path)

0it [00:00, ?it/s]
